In [19]:
from datetime import datetime
from time import time
import os

import sys
path_to_snakeAI = "drive/My Drive/snakeAI"
if path_to_snakeAI not in sys.path:
  sys.path.append(path_to_snakeAI)
print(f"{path_to_snakeAI} is in sys.path" if path_to_snakeAI in sys.path else f"{path_to_snakeAI} is not in sys.path")

import tensorflow as tf
import tensorflow.keras as k

from Model.model_train import train_dqn
from Snake.variables import Status


drive/My Drive/snakeAI is in sys.path


In [20]:

IMAGE_SIZE = (84, 84)
IMAGE_SHAPE = (*IMAGE_SIZE, 3)

learning_rate = 1e-4

config = {
  "num_rolling_avg_sample": 10,
  "evaluate_each": 20,
  "save_models": True,
  "save_graphs": True,
  "gamma": .95,
  "epsilon_decay": 0.000005,
  "boundaries": False,
  "maze_width": 10,
  "image_size": IMAGE_SIZE,
  "epsilon": .975,
  "path_to_weights": "drive/My Drive/snakeAI/trainings/AtariNet/11Jul2020__170013540334/model",
  "comment": f"""
  Architecture: AtariNet
  Optimizer: Adam
  Hyperparameters:
    lr = {learning_rate}
  Reward:
      if snake.status == Status.DEAD:
        return -1000
    else:
        r = 0
        r += 10 if snake.steps_without_food == 1 else 0
        return r

  """
}

In [21]:
def AtariNet(input_shape, ):
  model = k.Sequential([
      k.layers.Input(input_shape ),
      k.layers.Conv2D(16, 8, strides=4, activation='relu'),
      k.layers.Conv2D(32, 4, strides=2, activation='relu'),
      k.layers.Flatten(),
      k.layers.Dense(256, activation='relu'),
      k.layers.Dense(4, activation='linear')
  ])

  model.build(input_shape=input_shape)
  return model

In [22]:
def reward(snake, env, direction):
    if snake.status == Status.DEAD:
        return -1000
    else:
        r = 0
        r += 10 if snake.steps_without_food == 1 else 0
        return r

In [23]:
t = time()
config["training_dir"] = os.path.join(*['drive', 'My Drive', 'snakeAI', 'trainings', "AtariNet", datetime.now().strftime('%d%h%Y__%H%M%S%f')])
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model = AtariNet(input_shape=IMAGE_SHAPE, )
model.build(input_shape=(None, *IMAGE_SHAPE))
model.compile(optimizer=optimizer)

model = train_dqn(model, optimizer, reward, **config)
t = time() - t

____________________________________________________________________________________________


ValueError: ignored